<a href="https://colab.research.google.com/github/tpezzin/rna/blob/main/RedeNeural_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Baixei o arquivo compactado disponibilizado em https://github.com/fboldt/aulasann/blob/main/mnist.zip, coloquei no meu drive já descompactado e segui com a utilização do dataset

In [1]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [3]:
training_images_filepath = '/mydrive/RNA/MNIST/train-images.idx3-ubyte'
training_labels_filepath = '/mydrive/RNA/MNIST/train-labels.idx1-ubyte'
test_images_filepath = '/mydrive/RNA/MNIST/t10k-images.idx3-ubyte'
test_labels_filepath = '/mydrive/RNA/MNIST/t10k-labels.idx1-ubyte'

In [4]:
from mlxtend.data import loadlocal_mnist

train_images, train_labels = loadlocal_mnist( training_images_filepath, training_labels_filepath)
test_images, test_labels = loadlocal_mnist(test_images_filepath, test_labels_filepath)

In [5]:
(train_images.shape, train_labels), (test_images.shape, test_labels)

(((60000, 784), array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)),
 ((10000, 784), array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)))

In [6]:
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin

class DividePor255(BaseEstimator, TransformerMixin):
  def fit(self, X, y):
    return self
  def transform(self, X, y=None):
    return X.astype("float32") / 255

In [7]:
from sklearn.pipeline import Pipeline
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
import numpy as np

class RedeNeural(BaseEstimator, ClassifierMixin):
  def __init__(self, epochs=5, batch_size=128):
    self.epochs = epochs
    self.batch_size = batch_size

  def fit(self, X, y):
    self.labels, ids = np.unique(y, return_inverse=True)
    yhot = keras.utils.to_categorical(ids)
    self.model = keras.Sequential([
        layers.Conv2D(4, 4, activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(yhot.shape[1], activation="softmax")
    ])
    self.model.compile(optimizer="rmsprop",
                       loss="categorical_crossentropy",
                       metrics=["accuracy"])
    self.model.fit(X, yhot, epochs=self.epochs, batch_size=self.batch_size)
    return self

  def predict(self, X, y=None):
    probabilities = self.model.predict(X)
    ypred = self.labels[np.argmax(probabilities, axis=1)]
    return ypred

class MudaShape(BaseEstimator, TransformerMixin):
  def fit(self, X, y):
    return self
  def transform(self, X, y=None):
    return X.reshape((-1,28,28,1))

modelo = Pipeline([
    ("scaler", DividePor255()), 
    ("reshape", MudaShape()), 
    ("ann", RedeNeural(10))
])
modelo.fit(train_images, train_labels)
ypred = modelo.predict(test_images)
accuracy_score(test_labels, ypred)

Epoch 1/10
469/469 [==============================] - 37s 76ms/step - loss: 0.2410 - accuracy: 0.9275
Epoch 2/10
469/469 [==============================] - 30s 64ms/step - loss: 0.0778 - accuracy: 0.9763
Epoch 3/10
469/469 [==============================] - 30s 63ms/step - loss: 0.0486 - accuracy: 0.9849
Epoch 4/10
469/469 [==============================] - 30s 63ms/step - loss: 0.0331 - accuracy: 0.9897
Epoch 5/10
469/469 [==============================] - 29s 62ms/step - loss: 0.0230 - accuracy: 0.9927
Epoch 6/10
469/469 [==============================] - 29s 62ms/step - loss: 0.0151 - accuracy: 0.9954
Epoch 7/10
469/469 [==============================] - 30s 64ms/step - loss: 0.0105 - accuracy: 0.9969
Epoch 8/10
469/469 [==============================] - 32s 68ms/step - loss: 0.0068 - accuracy: 0.9980
Epoch 9/10
469/469 [==============================] - 29s 62ms/step - loss: 0.0055 - accuracy: 0.9983
Epoch 10/10
313/313 [==============================] - 2s 7ms/step


0.9877